In [4]:
import numpy as np
import pandas as pd
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [2]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 

In [ ]:
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/kannada_dataset_2_train.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X=list(dataset2_filtered['text'])
y=list(dataset2_filtered['label'])

In [ ]:
dataset2_filtered.shape

(5874, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [ ]:
dataset1=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/kannada_dataset_1_train.csv")
X_train=list(dataset1['text'])
y_train=list(dataset1['label'])
dataset1.shape

(6217, 2)

In [ ]:
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/kannada_dataset_1_test.csv")
X_test=list(dataset2['text'])
y_test=list(dataset2['label'])
dataset2.shape
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [ ]:
dataset1_filtered = dataset1[dataset1['label'] != 2]
dataset2_filtered = dataset2[dataset2['label'] != 2]

# Create train and test DataFrames
train_df = pd.DataFrame({"text": dataset1_filtered['text'], "labels": dataset1_filtered['label']})
test_df = pd.DataFrame({"text": dataset2_filtered['text'], "labels": dataset2_filtered['label']})

In [ ]:
train_df.shape

(4111, 2)

In [ ]:
test_df.shape

(1763, 2)

In [7]:
import torch
torch.cuda.device_count()

1

In [8]:
device = torch.cuda.get_device_properties(0)  # Assuming you have at least one GPU
print(device)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [6]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

# Kannada_BERT

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 15,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 5,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 3e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

kbert_model_kannada = ClassificationModel(
    "bert",
    "l3cube-pune/kannada-bert",
    num_labels=2,
    args=train_args,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/kannada-bert and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
kbert_model_kannada.train_model(train_df)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 3 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

...

Running Epoch 12 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 15 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

(975, 0.343020520473902)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = kbert_model_kannada.eval_model(test_df)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
print(result) #result of the model

{'mcc': 0.5104871194957058, 'accuracy': 0.8258650028360749, 'f1_score': 0.6128625472887768, 'tp': 243, 'tn': 1213, 'fp': 101, 'fn': 206, 'auroc': 0.8309459885488808, 'auprc': 0.6536273896359591, 'eval_loss': 0.5203070044517517}


In [ ]:
predictions, raw_outputs = kbert_model_kannada.predict(list(test_df['text'].values))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.92      0.85      0.89      1419
           1       0.54      0.71      0.61       344

    accuracy                           0.83      1763
   macro avg       0.73      0.78      0.75      1763
weighted avg       0.85      0.83      0.83      1763



In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8258650028360749


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8176815934571995


In [ ]:
import pickle
# save the model to disk
filename = 'kbert_kannada.sav'
pickle.dump(kbert_model_kannada, open(filename, 'wb'))

In [ ]:
# load the model from disk
import pickle
filename = 'kbert_kannada.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/kbert_kannada.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Kannada/kbert_kannada.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Kannada/kbert_kannada.sav'

In [9]:
import pickle
filename = '/content/drive/MyDrive/Capstone/Models/Kannada/kbert_kannada.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [10]:
sample_text="guru deshgaande swantha picture tegeyo sule maga alla yaav picture remake idhuthinking face " #1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [11]:
sample_text="ಗುರು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರೆ ಹಾಕು ಗುರು" #0
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0])

In [12]:
sample_text="ಏನ್ ತುಕಾಲಿ ಫ್ಯಾನ್ಸ್ ಮರ್ರೆ ಟ್ರೈಲರ್ ಕಾಮೆಂಟ್ ಮಾಡೋದು ಬಿಟ್ಟು ಫ್ಯಾನ್ಸ್ ಕಡೆಯಿಂದ ಅಲ್ ಬೆಸ್ಟ್ ಫ್ಯಾನ್ ಕಡೆಯಿಂದ ಆಲ್ ಬೆಸ್ಟ್ ಥು ಇವ್ರ ಜನ್ಮಕ್ಕೆ " #1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [13]:
sample_text="ಗುರು ನಿನ್ ಹೇಳು ಯಾರು ಮಾವಿನ್ ಸುಪೋರ್ಟ್ ಮಾಡ್ಬಾರ್ದು " #1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [16]:
sample_text="ಥಿಕ ಮುಚ್ಕೊಂಡು ಹೋಗು" #1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [17]:
sample_text="ಹೋಗೊ ಬೊಳಿ ಮಗನೇ" #1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])